In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
names = pd.read_table('public_dat\mimic_synthetic_feat.name',sep='\n',header=None)

In [4]:
name_col = names[0].tolist()
name_col

['SUBJECT_ID',
 'HADM_ID',
 'ADMITTIME',
 'DISCHTIME',
 'INSURANCE',
 'LANGUAGE',
 'RELIGION',
 'MARITAL_STATUS',
 'ETHNICITY',
 'GENDER',
 'AGE',
 'HR_MIN_DAY1',
 'HR_MAX_DAY1',
 'HR_MEAN_DAY1',
 'SYS_BP_MIN_DAY1',
 'SYS_BP_MAX_DAY1',
 'SYS_BP_MEAN_DAY1',
 'DIAS_BP_MIN_DAY1',
 'DIAS_BP_MAX_DAY1',
 'DIAS_BP_MEAN_DAY1',
 'MEAN_BP_MIN_DAY1',
 'MEAN_BP_MAX_DAY1',
 'MEAN_BP_MEAN_DAY1',
 'RESP_RATE_MIN_DAY1',
 'RESP_RATE_MAX_DAY1',
 'RESP_RATE_MEAN_DAY1',
 'TEMP_MIN_DAY1',
 'TEMP_MAX_DAY1',
 'TEMP_MEAN_DAY1',
 'SPO2_MIN_DAY1',
 'SPO2_MAX_DAY1',
 'SPO2_MEAN_DAY1',
 'GLUCOSE_MIN_DAY1',
 'GLUCOSE_MAX_DAY1',
 'GLUCOSE_MEAN_DAY1',
 'HR_MIN_DAY2',
 'HR_MAX_DAY2',
 'HR_MEAN_DAY2',
 'SYS_BP_MIN_DAY2',
 'SYS_BP_MAX_DAY2',
 'SYS_BP_MEAN_DAY2',
 'DIAS_BP_MIN_DAY2',
 'DIAS_BP_MAX_DAY2',
 'DIAS_BP_MEAN_DAY2',
 'MEAN_BP_MIN_DAY2',
 'MEAN_BP_MAX_DAY2',
 'MEAN_BP_MEAN_DAY2',
 'RESP_RATE_MIN_DAY2',
 'RESP_RATE_MAX_DAY2',
 'RESP_RATE_MEAN_DAY2',
 'TEMP_MIN_DAY2',
 'TEMP_MAX_DAY2',
 'TEMP_MEAN_DAY2',
 'SPO2_M

In [5]:
data = pd.read_table('public_dat\mimic_synthetic_train.data',sep=' ',header=None)

In [6]:
data.drop(0,axis=1,inplace=True)

In [7]:
data.columns = name_col

In [8]:
#data

In [9]:
died = pd.read_table('public_dat\mimic_synthetic_train.solution',sep='\n',header=None)


In [10]:
died.columns = ['DIED']

In [11]:
data = pd.concat([data,died],axis=1)

In [16]:
data.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)

In [17]:
#data.head()

In [18]:
num_cols = []
categorical_col = []

for cols in data.columns:
    if data[cols].dtype == object:
        categorical_col.append(cols)
    else:
        num_cols.append(cols)

In [15]:
# print('length of numerical columns',len(num_cols))
# print('length of categorical columns',len(categorical_col))

In [19]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
data[num_cols] = imputer.fit_transform(data[num_cols])

In [20]:
data.isnull().sum().sum()

3

In [21]:
null_counts = data.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

ICU               2
MARITAL_STATUS    1
dtype: int64

In [22]:
data["MARITAL_STATUS"].fillna("MARRIED", inplace = True) 
data["ICU"].fillna("MICU", inplace = True)  

In [23]:
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'AGE' has 74' unqiue categories
Feature 'HR_MIN_DAY1' has 79365' unqiue categories
Feature 'HR_MAX_DAY1' has 79462' unqiue categories
Feature 'HR_MEAN_DAY1' has 79298' unqiue categories
Feature 'SYS_BP_MIN_DAY1' has 79329' unqiue categories
Feature 'SYS_BP_MAX_DAY1' has 79246' unqiue categories
Feature 'SYS_BP_MEAN_DAY1' has 79362' unqiue categories
Feature 'DIAS_BP_MIN_DAY1' has 79511' unqiue categories
Feature 'DIAS_BP_MAX_DAY1' has 79393' unqiue categories
Feature 'DIAS_BP_MEAN_DAY1' has 79387' unqiue categories
Feature 'MEAN_BP_MIN_DAY1' has 79409' unqiue categories
Feature 'MEAN_BP_MAX_DAY1' has 79512' unqiue categories
Feature 'MEAN_BP_MEAN_DAY1' has 79138' unqiue categories
Feature 'RESP_RATE_MIN_DAY1' has 79552' unqiue categories
Feature 'RESP_RATE_MAX_DAY1' has 79563' unqiue categories
Feature 'RESP_RATE_MEAN_DAY1' has 79505' unqiue categories
Feature 'TEMP_MIN_DAY1' has 75975' unqiue categories
Feature 'TEMP_MAX_DAY1' has 75272' unqiue categories
Feature 'TEMP_MEAN_DA

Feature 'Anemia' has 2' unqiue categories
Feature 'Hepatitis' has 2' unqiue categories
Feature 'Acutp-hanm' has 2' unqiue categories
Feature 'Sicklecell' has 1' unqiue categories
Feature 'Coag/hemrdx' has 2' unqiue categories
Feature 'Whtblooddx' has 2' unqiue categories
Feature 'Othematldx' has 1' unqiue categories
Feature 'Adjustmentdisorders' has 1' unqiue categories
Feature 'Anxietydisorders' has 2' unqiue categories
Feature 'Attention-deficit/conduct/disruptivebeha' has 1' unqiue categories
Feature 'Delirium/dementia/amnestic/othercognitiv' has 2' unqiue categories
Feature 'Developmentaldisorders' has 1' unqiue categories
Feature 'Disordersusuallydiagnosedininfancy/ch' has 1' unqiue categories
Feature 'ImpulsecontroldisordersNEC' has 1' unqiue categories
Feature 'Mooddisorders' has 2' unqiue categories
Feature 'Personalitydisorders' has 1' unqiue categories
Feature 'Schizophreniaandotherpsychoticdisorde' has 1' unqiue categories
Feature 'Alcohol-relateddisorders' has 2' unqiue cat

In [24]:
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'INSURANCE' has 5' unqiue categories
Feature 'MARITAL_STATUS' has 7' unqiue categories
Feature 'ETHNICITY' has 41' unqiue categories
Feature 'GENDER' has 2' unqiue categories
Feature 'ICU' has 5' unqiue categories


In [25]:
#data.head()

In [26]:
#pip install imblearn

imblearn.over_sampling._smote.base.SMOTE

In [33]:
#trialss
# UnderSampled Balanced Dataset
from imblearn.over_sampling import SMOTE                        # Balance imbalance data
sm = SMOTE(random_state=2)
X, y = sm.fit_resample(X, np.ravel(y))
print(sum(y==0))
print(sum(y==1))

NameError: name 'X' is not defined

In [ ]:
data_copy=data

In [ ]:
# Separting categorical and numeric features
num_data_bal = balanced_data.select_dtypes(include = ['float64', 'int64'])
num_data_bal.shape

cat_data_bal = balanced_data.select_dtypes(include = ['O'])

print(num_data_bal.shape)
print(cat_data_bal.shape)

char_cols = cat_data_bal.columns
num_cols = num_data_bal.columns
target = num_data_bal["DIED"]

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt
def plot_pie(y):
    target_stats = Counter(y)
    labels = list(target_stats.keys())
    sizes = list(target_stats.values())
    explode = tuple([0.1] * len(target_stats))

    fig, ax = plt.subplots()
    ax.pie(sizes, explode=explode, labels=labels, shadow=True,
           autopct='%1.1f%%')
    ax.axis('equal')

In [ ]:
plot_pie(data["DIED"])  # Plotting the distribution of target variables

In [ ]:
plot_pie(balanced_data["DIED"])  # Plotting the distribution of target variables

In [ ]:
#correlation matrix for numeric data
corrmat = num_data_bal.corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'DIED')['DIED'].index
cm = np.corrcoef(num_data_bal[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

# The following plot shows that the correlation values with the dependent variable is too low

In [ ]:
## Correlation with dependent variable - First filter
corr_with_DIED = num_data_bal.corr()["DIED"].copy()
corr_with_DIED_abs = abs(corr_with_DIED).copy()  # Getting absolute of correlation values
corr_with_DIED_abs = corr_with_DIED_abs.sort_values(ascending = False) # Setting them in descending order
important_vars = [corr_with_DIED_abs.index[0]]
correlation_target_table = pd.DataFrame({'attribute':corr_with_DIED_abs.index, 'list':corr_with_DIED_abs.values})

# Selecting variables with >= 0.01 correlation with target variable 
correlation_target_selected_table = correlation_target_table[(correlation_target_table['list'] >= 0.01)]
important_variables = correlation_target_selected_table['attribute']
len(important_variables)

In [ ]:
important_variables

In [ ]:
# Drops the non-important variables
data_fin = num_data_bal[important_variables]

In [ ]:
data_fin.head()

In [ ]:
# place the DataFrames side by side
fb_data = pd.concat([cat_data_bal, data_fin], axis=1) 
print(fb_data.shape)  


In [ ]:
fb_data.head()

In [ ]:
X=data.iloc[:,:-1].values
y=data.iloc[:, -1].values

#Splitting into training and test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
X.shape, y.shape, X_train.shape , X_test.shape

In [ ]:
data = pd.concat([data,died],axis=1)

In [ ]:
data = pd.get_dummies(data)

In [ ]:
from imblearn.over_sampling import SMOTE                        # Balance imbalance data
sm = SMOTE(random_state=2)
X, y = sm.fit_sample(X, np.ravel(y))
print(sum(y==0))
print(sum(y==1))